In [1]:
pip install hazm

     |████████████████████████████████| 316 kB 473 kB/s eta 0:00:01
     |████████████████████████████████| 1.4 MB 802 kB/s eta 0:00:01
     |████████████████████████████████| 233 kB 3.3 MB/s eta 0:00:01
  Created wheel for nltk: filename=nltk-3.3-py3-none-any.whl size=1394470 sha256=b2a220fc67984100926ff8bf29ce6f42037770d4fcbaf641a918f5179c09ba94
  Stored in directory: /home/hamed/.cache/pip/wheels/19/1d/3a/0a8c14c30132b4f9ffd796efbb6746f15b3d6bcfc1055a9346
  Created wheel for libwapiti: filename=libwapiti-0.2.1-cp38-cp38-linux_x86_64.whl size=167516 sha256=fd87a15cc2533afb4cad5648aa7245c4998ada8e1a6824587458bb9e3cd4a5d2
  Stored in directory: /home/hamed/.cache/pip/wheels/3c/d8/9f/59fd78b2b7d1e9ffcb68fb6de80c2e7c20b804c8cbc4d8fc23
Successfully built nltk libwapiti
  Attempting uninstall: nltk
    Found existing installation: nltk 3.5
    Uninstalling nltk-3.5:
      Successfully uninstalled nltk-3.5
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install urlextract

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install kafka-python

     |████████████████████████████████| 246 kB 465 kB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [57]:
from kafka import KafkaConsumer
from json import loads
from time import sleep
from json import dumps
from kafka import KafkaProducer
import requests
import time
import re
import json

from urlextract import URLExtract
import json
import pandas as pd
from hazm import *
from sklearn.feature_extraction.text import TfidfVectorizer

tagger = POSTagger(model='/mnt/d/BigData/resources/postagger.model')


In [113]:
special_keywords = ['بورس','اقتصاد','تحریم','دولت','حسن روحانی','انتخابات','دلار','طلا','کرونا','تورم','دانشگاه','کووید۱۹','کوید۱۹','کویید۱۹','کویید ۱۹','کوید ۱۹']
special_keywords = set(special_keywords)

def extract_verbs(tokenized_sentence):
    verb_list = set()
    for word,tag in tokenized_sentence:
        if tag=='V':
            verb_list.add(word)
    return list(verb_list)

def keyWordExtraction(doc):
    text=doc['content']
    normalizer = Normalizer()
    vectorizer = TfidfVectorizer(lowercase=False, preprocessor=normalizer.normalize, tokenizer=word_tokenize,
                                 stop_words=stopwords_list())
    tokenized_sentence = tagger.tag(word_tokenize(text.replace('#','')))
    verbs = extract_verbs(tokenized_sentence)
    for v in verbs: # Remove Verbs From Text
        text = text.replace(v,'')
    vectors = vectorizer.fit_transform([text])
    feature_names = vectorizer.get_feature_names()
    denselist = vectors.todense().tolist()
    df = pd.DataFrame(denselist, columns=feature_names)
    
    for cl in df.columns:
        if cl.replace('#','') in special_keywords:
            df[cl]=0
    df.sort_values(by=0, axis=1, ascending=False, inplace=True, na_position='last')
    max = df.values.size
    min = int(max * 0.6)
    keywords = list(df.keys()[min:max].values)
    keywords = [x.replace('#','') for x in keywords]
    doc['keywords']= keywords
    return doc


In [6]:
consumer = KafkaConsumer(
    'PreProcess',
     bootstrap_servers=['localhost:9092'],
     auto_offset_reset='earliest',
     enable_auto_commit=True,
     group_id='my-group',
     value_deserializer=lambda x: loads(x.decode('utf-8')))

In [86]:
extractor = URLExtract()
def preprocess(tweet):
    tweet['hashtags'] = re.findall(r"#(\w+)",tweet['content'])
    tweet['links']= extractor.find_urls(tweet['content'])
    tweet = keyWordExtraction(tweet)
    return tweet

In [114]:
producer = KafkaProducer(bootstrap_servers=['localhost:9092'],key_serializer=str.encode,
                         value_serializer=lambda x:dumps(x).encode('utf-8'))

In [116]:
for message in consumer:
    message = message.value
    tweet = preprocess(message)
    producer.send('persistence', key=tweet['id'],value=tweet)
